In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 3ed54005-e5ae-4583-9176-6827b8e4ca03, 84, Finished, Available, Finished)

In [8]:
import pandas as pd
import requests
from io import StringIO

url = "https://www.mof.go.jp/english/policy/jgbs/reference/interest_rate/historical/jgbcme_all.csv"
response = requests.get(url, timeout=30)
response.encoding = 'shift_jis'

df = pd.read_csv(StringIO(response.text), header=1, na_values="-",parse_dates=["Date"])
df.head(5)
# 10年国債利回りを抽出

StatementMeta(, 3ed54005-e5ae-4583-9176-6827b8e4ca03, 88, Finished, Available, Finished)

,Date,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,25Y,30Y,40Y
0,1974-09-24,10.327,9.362,8.830,8.515,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1,1974-09-25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
2,1974-09-26,10.340,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
3,1974-09-27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
4,1974-09-28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# ============================================================
# Cell 2: pandas DataFrame → Spark DataFrame → Delta table 書き込み
# ============================================================

from pyspark.sql.types import StructType, StructField, DateType, DoubleType, TimestampType, IntegerType

# --- 明示的スキーマ定義（設計書準拠）---
schema = StructType([
    StructField("date",        DateType(),      nullable=False),
    StructField("yield_10y",   DoubleType(),    nullable=False),
    StructField("ingested_at", TimestampType(), nullable=False),
    StructField("year",        IntegerType(),   nullable=False),
    StructField("month",       IntegerType(),   nullable=False),
])

# --- スキーマ定義と同じ順序でカラムを並べる（位置マッピングのずれを防止）---
df_ordered = df[["date", "yield_10y", "ingested_at", "year", "month"]]

# --- pandas → Spark DataFrame 変換 ---
spark_df = spark.createDataFrame(df_ordered, schema=schema)

# --- Delta table 書き込み（overwrite: 再実行時も冪等性を保証）---
(
    spark_df
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("year", "month")
    .option("overwriteSchema", "true")
    .saveAsTable(TABLE_NAME)
)

actual_count = spark.read.table(TABLE_NAME).count()
print(f"[完了] Delta table '{TABLE_NAME}' に {actual_count:,} 行を書き込みました")
print(f"パーティション: year / month")


In [ ]:


# pandas DataFrame → Spark DataFrame に変換し、Delta table として書き込み

spark_df = spark.createDataFrame(df)

table_name = "bronze_interest_rate"
spark_df.write.format("delta").mode("overwrite").saveAsTable(f"dbo.{table_name}")

print(f"Delta table '{table_name}' に {spark_df.count()} 行を書き込みました")

StatementMeta(, 3ed54005-e5ae-4583-9176-6827b8e4ca03, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4aee22ec-fa7e-4fa1-876c-9a53000b7a7d)

In [ ]:
#---型変換---
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, DoubleType, IntegerType, TimestampType 
schema = StructType([
    StructField("Date", DateType(), True),
    StructField("10Y", DoubleType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("ingested_at", TimestampType(), True)])


StatementMeta(, 3ed54005-e5ae-4583-9176-6827b8e4ca03, 200, Finished, Available, Finished)

In [ ]:
TABLE_NAME = "bronze_interest_rate"
# --- pandas → Spark DataFrame 変換 ---
spark_df = spark.createDataFrame(df, schema=schema)

# --- Delta table 書き込み（overwrite: 再実行時も冪等性を保証）---
(
    spark_df
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("year", "month")
    .option("overwriteSchema", "true")
    .saveAsTable(TABLE_NAME)
)

actual_count = spark.read.table(TABLE_NAME).count()
print(f"[完了] Delta table '{TABLE_NAME}' に {actual_count:,} 行を書き込みました")
print(f"パーティション: year / month")


StatementMeta(, 3ed54005-e5ae-4583-9176-6827b8e4ca03, 201, Finished, Available, Finished)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported cast from double to timestamp using function cast_timestamp
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


PySparkTypeError: [CANNOT_ACCEPT_OBJECT_IN_TYPE] `IntegerType()` can not accept object `9.362` in type `float`.